# DeepASM

## Install packages

In [ ]:
# To manipulate HDF5 files (RUN FOR ALL MODELS)
!pip3 install --upgrade tables

# Install Decision Forest models
!pip3 install tensorflow_decision_forests==0.2.2


## Import packages

In [ ]:
import sys

# Python packages for data, stats, and visualization
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns 
import random

# Machine learning libraries
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.models import load_model
from tensorflow.keras import layers
import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Kernel functions
from sklearn.neighbors import KernelDensity
from numpy import asarray
from matplotlib import pyplot
from numpy import exp

# Dimensionality reduction
from sklearn.decomposition import PCA, KernelPCA, NMF, TruncatedSVD
from sklearn.manifold import TSNE, LocallyLinearEmbedding, SpectralEmbedding

# To get the time
from datetime import datetime

# To write on the same line
from IPython.display import clear_output

# Decision tree algorithms
import tensorflow_decision_forests as tfdf
 
# Figure parameters
mpl.rcParams['figure.figsize'] = (10, 10)
mpl.rcParams['axes.titlesize'] = 15
mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
# Print different versions
print(sys.version)
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("Numpy version:", np.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## GCP Variables

In [ ]:
# Import raw data from bucket. False if you want to import the processed dataset
IMPORT_RAW_FROM_BUCKET = True

# Export data after it's been prepared
EXPORT_PROCESSED_DATA = True

# Import data after its features have been prepared
IMPORT_PROCESSED_DATA = False
PROCESSED_DATA_PATH = "deepasm/notebook/250bp_500000rows_2022-03-20_23-24-20"

# Bucket name where the training datasets are
DEEPASM_BUCKET="deepasm"


## Dataset used for training

In [ ]:
# Run the training on a small sample size (50k)
TEST_RUN = True

# Size of the genomic window (250bp, 500bp, 1000bp)
GENOMIC_INTERVAL = 250

# Number of rows to take into the dataset after import
if TEST_RUN == True:
    NB_ROWS_RAW_DATASET = int(500000) # The maximum is 5e6. We use 200k to test the code
else:
    NB_ROWS_RAW_DATASET = int(5e6) 

## Model variables

In [ ]:
#--------------------------------------------------
# Gradient-Boosted algorithms

GROWING_STRATEGY = "BEST_FIRST_GLOBAL"  # LOCAL (default). or BEST_FIRST_GLOBAL is the default
NUM_TREES = 600 # Maxmimum number of decision trees. (default 300)
MIN_EXAMPLES = 10 # Minimum number of examples in a node (default: 5)
MAX_DEPTH = 12 # Maximum depth of the tree (default 6)
SUBSAMPLE = 0.5 # Ratio of the dataset (sampling without replacement) used to train individual trees for the random sampling method (Default 1)
SAMPLING_METHOD = "RANDOM"

#--------------------------------------------------
# Parameters common to all models

# Minimum correlation factor. Under that, remove features
MIN_CORR = 0.05

# Kernel values for probability estimates
KERNEL_FM_NB_VALUES = 10
KERNEL_FM_BANDWIDTH = 0.1
KERNEL_COV_NB_MAX = 200
KERNEL_COV_NB_STEP = 40
KERNEL_COV_BANDWIDTH = 20

# Early stopping
EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=0,
    patience=10,
    mode='auto',
    restore_best_weights=True)

# Percentage of data points to be used in the Test dataset
TEST_SPLIT = 0.2

# Percentage of datapoints used between training and validation
VALIDATION_SPLIT = 0.3 # How to divide the training dataset for validation

EPOCHS = 100 # We have so many datapoints that 20 epochs are enough to stabilize the training
BATCH_SIZE = 1000 # to get a few identified ASM we need at a few hundreds since the
# frequency of ASM is 1.38%
# A batch size of 1000 will run into a memory error on TF 2.7

# Regularlization L1 and L2 (defaults are l1 = 0.01 and l2 = 0.01)
L1_R = 0
L2_R = 1e-3

#--------------------------------------------------
# Parameters common to neural network models
ACTIVATION_FUNCTION = 'tanh' # 'tanh' # or 'relu' or 'gelu (Gaussian Error Linear Unit)'
NB_NODES_PERCEPTRON = 60
NB_LAYERS_PERCEPTRON = 5
NB_NODES_AFTER_CNN = 2

# CNN parameters
CNN_FILTERS = 16
CNN_KERNEL = 100 # Must be smaller than the genomic region (250). The av distance between CpG is 37 bp and the std dev of the distances between cpgs is 24 bp
LEARNING_RATE = 3e-4 

# Learning rate was taken from this
# http://karpathy.github.io/2019/04/25/recipe/#2-set-up-the-end-to-end-trainingevaluation-skeleton--get-dumb-baselines

#--------------------------------------------------
# Parameters common to RNN

RNN_UNITS = 128 # 64 orginally

#--------------------------------------------------
# SPECIFIC TO RANDOM FOREST ALGORITHM
use_raw_df_for_forest_models = False

## ML evaluation metrics

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='sensitivity'),
      keras.metrics.AUC(name='auc')
      ]

def plot_metrics(history):
  metrics =  ['loss', 'auc', 'precision', 'sensitivity']
  plt.figure(figsize=(10,10))
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.ylim([0,1])

    plt.legend()


def display_results(df_results):
  print("Loss", np.round(df_results[0], 3))
  print("True positives", np.round(df_results[1], 3))
  print("False positives", np.round(df_results[2], 3))
  print("True negatives", np.round(df_results[3], 3))
  print("False negatives", np.round(df_results[4], 3))
  print("Accuracy", np.round(df_results[5], 3))
  print("Precision", np.round(df_results[6], 3))
  print("Sensitivity", np.round(df_results[7], 3))
  print("AUC", np.round(df_results[8], 3))

def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,80])
  plt.ylim([0,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')



## Import raw data

In [ ]:
if IMPORT_RAW_FROM_BUCKET == True:
    !gsutil ls gs://$DEEPASM_BUCKET/$GENOMIC_INTERVAL*bp/encode_training_data_with_genomic_picture/*.json > list_to_download.txt
    files_to_download_df = pd.read_csv('list_to_download.txt', header=None)
    print("Number of files to download:", files_to_download_df.shape[0])

    imported_df = pd.DataFrame()
    
    if TEST_RUN == True:
        range_files = 1
    else: # Download all the files
        range_files = files_to_download_df.shape[0]

    for index_file in range(range_files): 
        file_name_bucket = files_to_download_df[0][index_file]
        local_file_name = "training_" + str(index_file) + ".json"
        
        # Download the file from bucket
        !gsutil cp $file_name_bucket $local_file_name
        
        print("Appending file...")
        imported_df = imported_df.append(pd.read_json(local_file_name, lines = True))

In [ ]:
print("Size of the imported dataset:", imported_df.shape)

In [ ]:
#imported_df['nb_reads'].describe()
imported_df

## Prepare the features

Note: we do not randomize the rows because the scripts preceding this notebook already sampled the rows.

### Copy & clean dataframe 

In [ ]:
# # Copy of the dataframe
# raw_df = imported_df.copy()
# raw_df = raw_df.head(NB_ROWS_RAW_DATASET)
# # Randomize the rows
# raw_df = raw_df.sample(frac = 1, ignore_index = True)

# USED FOR TESTING
df_with_asm = imported_df[imported_df['asm_snp'] == 1].head(20).reset_index(drop=True)
df_without_asm = imported_df[imported_df['asm_snp'] == 0].head(20).reset_index(drop=True)
                     
raw_df = df_with_asm.append(df_without_asm).reset_index(drop=True)                     



In [ ]:
print("Size of the dataset: ", raw_df.shape)
print("Number of samples (12 expected):", len(raw_df['sample'].unique()))
print("Chromosomes found (24 expected):", raw_df['chr'].unique())

# We remove the sample column
raw_df = raw_df.drop('sample', axis = 1)

# We remove chromosomes X and Y because ASM cannot be reliabily evaluated in these chromosomes
raw_df = raw_df[raw_df['chr'] != 'X']
raw_df = raw_df[raw_df['chr'] != 'Y']

# We remove the chr column
raw_df = raw_df.drop('chr', axis = 1)

In [ ]:
# TEMPORARY
raw_df = raw_df[['asm_snp', 'nb_reads', 'nb_cpg_found',  'genomic_picture', 'cpg_cov', 'cpg_pos', 'cpg_fm']].reset_index(drop=True)
raw_df
#raw_df['genomic_picture'][0]


### Calculate the distance between CpGs

In [ ]:
# Create a function to calculate the distance between CpGs (~3min)
def dist_cpg(cpg_pos):
  distances = []
  for index in range(len(cpg_pos)):
    if index >= len(cpg_pos)-1:
      return distances
    else:
      distances.append(cpg_pos[index + 1] - cpg_pos[index])
  return distances

# Apply the function "distance" to the array of CpG positions
raw_df['cpg_dist'] = raw_df['cpg_pos'].apply(lambda x: dist_cpg(x))

### Convert arrays into numerical features

To do this, we use kernel estimates as well as simpler metrics like mean and standard deviation

#### Kernel functions

In [ ]:
# FRACTIONAL METHYLATION

# Values for fractional methylation (between 0 and 1)
values_for_kernel_fm = asarray([value for value in range(0, KERNEL_FM_NB_VALUES+1)])
values_for_kernel_fm = values_for_kernel_fm / KERNEL_FM_NB_VALUES
print("X-axis values used for the FM kernel estimate:", values_for_kernel_fm)
values_for_kernel_fm = values_for_kernel_fm.reshape((len(values_for_kernel_fm), 1))

# Build Kernel model
kernel_fm_model = KernelDensity(bandwidth=KERNEL_FM_BANDWIDTH, kernel='gaussian')

# Function to be applied to each array in the columns read_fm and cpg_fm
def estimate_kernels_fm(x):
  sample = np.reshape(x, (len(x), 1))
  kernel_fm_model.fit(sample)
  probabilities = kernel_fm_model.score_samples(values_for_kernel_fm)
  probabilities = exp(probabilities)
  return np.round(probabilities, 4)

# Try function
#estimate_kernels_fm(raw_df['cpg_fm'][0])


In [ ]:
# COVERAGE AND CPG DISTANCE

# Values for fractional methylation (between 0 and 1)
values_for_kernel_cov = asarray([value for value in range(0, KERNEL_COV_NB_MAX, KERNEL_COV_NB_STEP)])
print("Values used in kernel estimate:", values_for_kernel_cov)
values_for_kernel_cov = values_for_kernel_cov.reshape((len(values_for_kernel_cov), 1))

# Build Kernel model
kernel_cov_model = KernelDensity(bandwidth=KERNEL_COV_BANDWIDTH, kernel='gaussian')

# Function to be applied to each array in the columns read_fm and cpg_fm
def estimate_kernels_cov(x):
  sample = np.reshape(x, (len(x), 1))
  kernel_fm_model.fit(sample)
  probabilities = kernel_fm_model.score_samples(values_for_kernel_cov)
  probabilities = exp(probabilities)
  return np.round(probabilities, 4)

# Try function
#estimate_kernels_cov(raw_df['cpg_cov'][1])

#### Test kernel estimates

In [ ]:
variable_to_plot = 'cpg_fm' # cpg_fm or read_fm or cpg_dist or cpg_cov

n_extract = 10
extract_asm = raw_df[raw_df['asm_snp'] == 1].sample(n=n_extract, ignore_index = True)
extract_noasm = raw_df[raw_df['asm_snp'] == 0].sample(n=n_extract, ignore_index = True)
n_x = round(np.sqrt(n_extract))

##### Plots for regions with ASM

In [ ]:
mpl.rcParams['figure.figsize'] = (10, 10)
fig, axs = plt.subplots(n_x, n_x, sharey=True, sharex=True, tight_layout=True)

for k in range(n_x):
  for m in range(n_x):

    # Print data distribution
    data_distribution = extract_asm[variable_to_plot][k+m]
    axs[k,m].hist(data_distribution, density = True, bins = 10)

    # Print kernel density
    if 'fm' in variable_to_plot:
        #print("Using the FM kernel estimates")
        kernel_probabilities = estimate_kernels_fm(data_distribution)
        values = values_for_kernel_fm
    else:
        #print("Using the COV kernel estimates")
        kernel_probabilities = estimate_kernels_cov(data_distribution)
        values = values_for_kernel_cov
    axs[k,m].plot(values[:], kernel_probabilities)


##### Plots for regions without ASM

In [ ]:
mpl.rcParams['figure.figsize'] = (10, 10)
fig, axs = plt.subplots(n_x, n_x, sharey=True, sharex=True, tight_layout=True)

for k in range(n_x):
  for m in range(n_x):

    # Print data distribution
    data_distribution = extract_noasm[variable_to_plot][k+m]
    axs[k,m].hist(data_distribution, density = True, bins = 10)

    # Print kernel density
    if 'fm' in variable_to_plot:
        #print("Using the FM kernel estimates")
        kernel_probabilities = estimate_kernels_fm(data_distribution)
        values = values_for_kernel_fm
    else:
        #print("Using the COV kernel estimates")
        kernel_probabilities = estimate_kernels_cov(data_distribution)
        values = values_for_kernel_cov
    axs[k,m].plot(values[:], kernel_probabilities)

#### Calculate the mean, std, and kernel estimates of arrays

In [ ]:
def convert_arrays(df, column_name):
  """Inputs: dataframe and a column name that contains arrays"""

  # Mean and Standard deviation
  std_name = "std_" + column_name
  av_name = "mean_" + column_name

  print("Calculating the standard deviation")
  df[std_name] = df[column_name].apply(lambda x: np.round(np.std(x), 4))
  print("Calculating the average")
  df[av_name] = df[column_name].apply(lambda x: np.round(np.mean(x), 4))
  
  # Kernel density estimates
  kernel_name = "kernel_" + column_name
  if (column_name == 'cpg_cov' or column_name == 'cpg_dist'):
    print("Calculating the proba distribution for cov or dist")
    df[kernel_name] = df[column_name].apply(lambda x: estimate_kernels_cov(x))
  else:
    print("Calculating the proba distribution for fractional methylation")
    df[kernel_name] = df[column_name].apply(lambda x: estimate_kernels_fm(x))


In [ ]:
# Apply the function
for col in ['read_fm', 'cpg_fm', 'cpg_dist']:
    clear_output(wait=True)
    print("Column: ", col)
    convert_arrays(raw_df, col)
print("DONE")

In [ ]:
def export_kernel_array(col):
    # Col must be a column of kernel estimate arrays
    print("Processing:", col)
    kernel_name_list = []
    if 'fm' in col:
        values = values_for_kernel_fm
    else:
        values = values_for_kernel_cov
    # Create a list of the new column names
    for k in range(0, values.shape[0]):
        kernel_name = col + "_kernel_" + str(k)
        kernel_name_list = kernel_name_list + [kernel_name]
    print(kernel_name_list)
    
    # Create the additional columns
    kernel_estimates_column = "kernel_" + col
    raw_df[kernel_name_list] = pd.DataFrame(raw_df[kernel_estimates_column].tolist(), index= raw_df.index)

In [ ]:
# Use the function
for col in ['read_fm', 'cpg_fm', 'cpg_dist']:
    export_kernel_array(col)

In [ ]:
# Delete columns that we no longer need
for col in ['read_fm', 'cpg_dist',
            'kernel_cpg_dist', 'kernel_cpg_fm', 'kernel_read_fm']:
    raw_df.drop(col, axis = 1, inplace = True)

### Convert epigenetic signals into dummy variables

#### Plot histograms for the values of epigenetic signals

In [ ]:
#df_extract = raw_df[['dnase','encode_ChiP_V2', 'tf_motifs']]
#df_extract
#hist = raw_df[['dnase','encode_ChiP_V2', 'tf_motifs']].hist(density = True, bins = 3)

sns.set(rc = {'figure.figsize':(12,12)})
#sns.pairplot(raw_df[['asm_snp','dnase','encode_ChiP_V2', 'tf_motifs']], hue = 'asm_snp', diag_kind='kde')

In [ ]:
raw_df.columns

In [ ]:
def convert_epi_signal(epi_signal):
  print("Processing signal", epi_signal)
  unique_values = raw_df[epi_signal].unique()
  print(unique_values)
  min_epi_value = 0 # It's always zero (no signal) for all signals
  median_epi_value = np.median(unique_values)
  print("Median epi value:", median_epi_value)
  epi_signal_null = epi_signal + "_null"
  epi_signal_low = epi_signal + "_low"
  epi_signal_high = epi_signal + "_high"
  raw_df[epi_signal_null] = raw_df[epi_signal].apply(lambda x: 1 if x == min_epi_value else 0)
  raw_df[epi_signal_low] = raw_df[epi_signal].apply(lambda x: 1 if (x > min_epi_value and x <= median_epi_value) else 0)
  raw_df[epi_signal_high] = raw_df[epi_signal].apply(lambda x: 1 if x > median_epi_value else 0)

In [ ]:
# Apply the function to all epigenetic signals
for epi_signal in ['dnase', 'encode_ChiP_V2', 'tf_motifs']:
  convert_epi_signal(epi_signal)

In [ ]:
# Delete the raw epigenetic signals
# for epi_signal in ['dnase', 'encode_ChiP_V2', 'tf_motifs']:
#   raw_df.drop(epi_signal, axis = 1, inplace = True)

In [ ]:
raw_df.columns

## Create a 2D image of the genomic region (20 reads x 3 consecutive CpGs)

We select 3 CpGs whose FM is between 0.4 and 0.6 if possible. Then we get all 20 reads that cover them all.

In [ ]:
def find_cpgs_for_pic(df_row, nb_cpg, min_cov_cpg):
    # df_row is a typical row of the raw_df dataset
    # We find the nb_cpg CpGs whose FM is the closest to 0.5
    
    # We first sort the FM by its distance from 0.5
    ranked_fm = [round(abs(x-0.5),3) for x in df_row['cpg_fm']]
    new_pos_sorted = [i for _, i in sorted(zip(ranked_fm, df_row['cpg_pos']))]
    new_cov_sorted = [i for _, i in sorted(zip(ranked_fm, df_row['cpg_cov']))]
    #print(ranked_fm)
    #print(new_pos_sorted)
    
    filtered_cpg = []
    
    for k in range(0, len(new_pos_sorted)):
        #print(new_cov_sorted[k])
        if new_cov_sorted[k] >= min_cov_cpg:
            #print("covered enough")
            filtered_cpg = filtered_cpg + [new_pos_sorted[k]]
    
    # print(ranked_fm)
    # print(new_pos_sorted)
    # print(new_cov_sorted)
    
    # We return the first 3 elements of the list and we sort them
    return sorted(filtered_cpg[:min(len(df_row['cpg_pos']),nb_cpg)])
        
# Quick example
find_cpgs_for_pic(raw_df.iloc[28], 3, 15)

#print("HI")
    

In [ ]:
# Apply the function to the dataframe
# Find 3 CpGs at least 10x covered as close as possible to a FM of 0.5
raw_df['cpgs_for_pic'] = raw_df.apply(lambda x: find_cpgs_for_pic(x, 3, 10), axis = 1)
raw_df

In [ ]:
# Drop the FM and CpG pos columns
raw_df.drop(['cpg_pos', 'cpg_fm', 'cpg_cov', 'nb_cpg_found'], inplace = True, axis = 1)
raw_df.head()

In [ ]:
#raw_df['genomic_picture'][0]

In [ ]:
# For a read, create a dictionary of 1/ read FM and 2/ array of methylation status
def find_methylation_array_from_one_read(genomic_dictionary, cpg_array):
    # genomic dictionary contains read ID, its FM, an array of CpG pos and an array of CpG methylation status
    # cpg_array is the list of 3 CpGs we need to find the methylation array for
    
    #print(genomic_dictionary)
    #print(cpg_array)
    
    # Initialize the methylation array
    meth_array = []
    
    # When importing the arrays of CpG pos and FM, there were converted into strings
    genomic_dictionary['pos_array'] = list(map(int, genomic_dictionary['pos_array']))
    genomic_dictionary['meth_array'] = list(map(int, genomic_dictionary['meth_array']))
        
    if int(genomic_dictionary['nb_cpg']) >= len(cpg_array): # no need to go through the dic if it does not cover at least 3 CpGs
        for k in range(0,len(cpg_array)):
            #print(cpg_array[k])
            if cpg_array[k] in genomic_dictionary['pos_array']:
                cpg_pos_in_array = genomic_dictionary['pos_array'].index(cpg_array[k])
                meth_array = meth_array + [genomic_dictionary['meth_array'][cpg_pos_in_array]]
        
        # Create a new dictionary to return
        new_dic = {'read_fm': round(genomic_dictionary['read_fm'],3), 
                   'meth_array': meth_array}
    else:
        new_dic = []
    
    # if did not find all CpGs
    if meth_array != []:
        if len(meth_array) < len(cpg_array):
            new_dic = []
    
    # If we did not find ANY CpG
    if meth_array == []:
        new_dic = []
    
    return new_dic
        
# Quick test
test = {'read_id': 'E00247:448:H3HNYCCXY:2:1219:8471:23970_1:N:0:TATAAT',
        'read_fm': 0.667,
        'nb_cpg': '4',
        'pos_array': ['3', '1', '9'],
        'meth_array': ['1', '0', '0']}
find_methylation_array_from_one_read(test, [3,9])

#### Quick test 2
#test1 = {'read_id': 'E00247:448:H3HNYCCXY:2:1201:11698:14441_1:N:0:TATAAT', 'read_fm': 0.8180000000000001, 'nb_cpg': '11', 'pos_array': [47, 115, 150, 157, 101, 175, 105, 138, 167, 113, 185], 'meth_array': [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]}
#test2 = [16, 200, 229]
#find_methylation_array_from_one_read(test1, test2)


In [ ]:
# Build the array (3, 20, 1)
def create_2d_pic(row_df, min_cov):
    
    array_of_new_dics = []
    
    # row_df is a row in the raw_df dataframe
    for k in range(0, len(row_df['genomic_picture'])):
        #print(row_df['genomic_picture'][k])
        #print(row_df['cpgs_for_pic'])
        #print(find_methylation_array_from_one_read(row_df['genomic_picture'][k], row_df['cpgs_for_pic']))
        #print("**************")
        array_of_new_dics = array_of_new_dics + [find_methylation_array_from_one_read(row_df['genomic_picture'][k], row_df['cpgs_for_pic'])]
    
    array_of_new_dics = [x for x in array_of_new_dics if x != []]
    print(array_of_new_dics)
    
#     # Remove empty lists
#     result = [x for x in result if x != []]
    
#     if np.any(result):
#         #print("Coverage:", len(result))
#         if len(result) >= min_cov:
            
#             # We sample 20 reads at random
#             result_min_cov = random.sample(result, min_cov)

#             # Sort the array by the read FM
#             sorted_result = sorted(result_min_cov, key = lambda d: d['read_fm'], reverse = True)
#             #sorted_result = result

#             # Create the array of arrays in the same order
#             genomic_2d_picture = []
#             for k in range(0, len(sorted_result)):
#                 genomic_2d_picture = genomic_2d_picture + [sorted_result[k]['meth_array']]

#             # Convert into array
#             genomic_2d_picture = np.array(genomic_2d_picture)
#         else:
#             #print("Not enough coverage")
#             genomic_2d_picture = []
        
#     else:
#         #print("EMPTY")
#         genomic_2d_picture = []
    
#     return genomic_2d_picture

# Quick test
result_test = create_2d_pic(raw_df.iloc[1], 3)
print(result_test)


## Create a 2D image of the genomic region (20 reads x 250 bp)

In [ ]:
def create_1d_genomic_window(genomic_dictionary, genomic_interval, nb_cpg_in_region):
    # genomic dictionary contains read ID, its FM, an array of CpG pos and an array of CpG methylation status
    # This creates an array of array. Its length is the length of the genomic window (250bp) and each array
    # contains [a,b] where a indicates if a CpG was found (0 or 1) and b is the methylation status of the CpG (0 or 1)
    
    # When importing the arrays of CpG pos and FM, there were converted into strings
    genomic_dictionary['pos_array'] = list(map(int, genomic_dictionary['pos_array']))
    genomic_dictionary['meth_array'] = list(map(int, genomic_dictionary['meth_array']))
    
    #print(genomic_dictionary)
    
    # We initialize the array
    result = []
    
    if int(genomic_dictionary['nb_cpg']) >= nb_cpg_in_region:
        for genomic_position in range(1,genomic_interval+1):
            #print(genomic_position)
            if genomic_position in genomic_dictionary['pos_array']:
                #print("Found this CpG on the read")
                cpg_pos_in_array = genomic_dictionary['pos_array'].index(genomic_position)
                result = result + [[1,genomic_dictionary['meth_array'][cpg_pos_in_array]]]
            else:
                result = result + [[0,0]]
            #print(result)
    
        # Convert list into array
        result = np.array(result)
        new_dic = {'read_id': genomic_dictionary['read_id'], 
                   'read_fm': genomic_dictionary['read_fm'], 
                   'nb_cpg': genomic_dictionary['nb_cpg'],
                    'genomic_1d_window': result}
    else:
        new_dic = []
    
    return new_dic


# Quick test
test = {'read_id': 'E00247:448:H3HNYCCXY:2:1219:8471:23970_1:N:0:TATAAT',
        'read_fm': 0.667,
        'nb_cpg': '3',
        'pos_array': ['1', '3', '9'],
        'meth_array': ['1', '1', '0']}

result_test = create_1d_genomic_window(test, 10, 3)
print(result_test)

In [ ]:
def create_2d_genomic_window(row_df, genomic_interval, min_cov):
    # row_df is a row in the raw_df data.
    # The genomic array is an array of genomic dictionaries (see function above).
    # This will create a 2D image: an array of of nested arrays. nested arrays are arrays of arrays. 
    # They are 250 length and contain [a,b] where a indicates if a CpG was found (0 or 1) and b is the methylation status of the CpG (0 or 1)
    # The number of 250-arrays is equal to the number of reads for that genomic window.
    
    genomic_array = row_df['genomic_picture']
    nb_cpg_in_region = row_df['nb_cpg_found']
    
    required_nb_cpg = round(0.3*nb_cpg_in_region)
    
    #print(nb_cpg_in_region)
    #print("Requiring", required_nb_cpg, " cpgs")
    
    # Create the genomic 1D window for each read in the array
    result = np.array([create_1d_genomic_window(x, genomic_interval, required_nb_cpg) for x in genomic_array])
    
    # Remove empty lists
    result = [x for x in result if x != []]
    #print(result.shape)
    #print("result", result)
    # If the result is not empty:
    if np.any(result):
        #print("Coverage:", len(result))
        if len(result) >= min_cov:
            result_min_cov = random.sample(result, min_cov)

            # Sort the array by the read FM
            sorted_result = sorted(result_min_cov, key = lambda d: d['read_fm'], reverse = True)
            #sorted_result = result

            # Create the array of arrays in the same order
            genomic_2d_picture = []
            for k in range(0, len(sorted_result)):
                genomic_2d_picture = genomic_2d_picture + [sorted_result[k]['genomic_1d_window']]

            # Convert into array
            genomic_2d_picture = np.array(genomic_2d_picture)
        else:
            #print("Not enough coverage")
            genomic_2d_picture = []
        
    else:
        #print("EMPTY")
        genomic_2d_picture = []
    
    return genomic_2d_picture

#tmp1 = {'read_id': 'read_id_121391', 'nb_cpg': '3', 'read_fm': 0.33, 'pos_array':[2,5,7], 'meth_array':[0,1,0]}
#tmp2 = {'read_id': 'read_id_12', 'nb_cpg': '3', 'read_fm': 0.66, 'pos_array':[1,2,3], 'meth_array':[1,1,0]}
#test = [tmp1, tmp2]
#print(type(result_test))

result_test = create_2d_genomic_window(raw_df.iloc[0], 10, 10)
#print("Final result", result_test)


In [ ]:
# Split the dataframe into a list of dataframes of XX rows. Otherwise it stalls
nb_rows_in_splits = 100
nb_dataframe_pieces = max(1, round(raw_df.shape[0]/nb_rows_in_splits))
raw_df_pieces = np.array_split(raw_df, nb_dataframe_pieces)
print("The dataframe has been split into", nb_dataframe_pieces, "pieces")

In [ ]:
# Apply the function to the dataframe which we split before because it takes a lot of memory.
for df_piece in range(nb_dataframe_pieces): # nb_dataframe_pieces
        clear_output(wait=True)
        print("processing the piece at position:", df_piece)
        raw_df_pieces[df_piece]['genomic_2d_picture'] = raw_df_pieces[df_piece].apply(lambda x: create_2d_genomic_window(x, GENOMIC_INTERVAL, 20), axis = 1)
        raw_df_pieces[df_piece] = raw_df_pieces[df_piece][raw_df_pieces[df_piece]['genomic_2d_picture'].str.len() > 0]

In [ ]:
# Concatenate the dataframes
prepared_df = pd.DataFrame()

for df_piece in range(nb_dataframe_pieces):
    clear_output(wait=True)
    print("processing the piece at position:", df_piece)
    tmp_df = raw_df_pieces[df_piece]
    tmp_df.drop('genomic_picture', axis = 1, inplace = True)
    prepared_df = prepared_df.append(tmp_df)

In [ ]:
prepared_df.shape
#prepared_df[prepared_df['genomic_2d_picture'].str.len() > 0]

## Create a 1D "image" of the genomic region

We create an 1D-image (length: genomic region interval) with 3 information per "pixel": CpG presence (0 or 1), CpG coverage, CpG fractional methylation

In [ ]:
# Create arrays of positions, fractional methylation, and coverage for CpGs (~2h30)
def create_genomic_array(df):
    genomic_positions = []
    genomic_fm = []
    for position in range(df['region_inf'], df['region_sup'] + 1):
        if position in df['cpg_pos']:
            new_pos = 1
            pos_index = df['cpg_pos'].index(position)
            new_fm = df['cpg_fm'][pos_index]
        else:
            new_pos = 0
            new_fm = 0
        genomic_positions = genomic_positions + [new_pos]
        genomic_fm = genomic_fm + [new_fm]
    return np.transpose([genomic_positions, genomic_fm])


In [ ]:
# Test function
d = {'cpg_pos': [[2],[21,22]], 
              'region_inf':[1,20], 
              'region_sup':[4,23],
              'cpg_fm': [[0.0],[1.0,0.0]]
             }
example_df = pd.DataFrame(data = d)
example_df['genomic_pic'] = example_df.apply(lambda x: create_genomic_array(x), axis = 1)
example_df

### Split the dataset for calculating the images

In [ ]:
# Split the dataframe into a list of dataframes of 10 rows. Otherwise it stalls
nb_dataframe_pieces = max(1, round(raw_df.shape[0]/10))
raw_df_pieces = np.array_split(raw_df, nb_dataframe_pieces)
print("The dataframe has been split into", nb_dataframe_pieces, "pieces")


In [ ]:
# Apply the function to the dataframe which we split before because it takes a lot of memory.
for df_piece in range(nb_dataframe_pieces): # nb_dataframe_pieces
        clear_output(wait=True)
        print("processing the piece at position:", df_piece)
        raw_df_pieces[df_piece]['genomic_matrix'] = raw_df_pieces[df_piece].apply(lambda x: create_genomic_array(x), 
                                                                                      axis = 1)

In [ ]:
# Concatenate the dataframes
prepared_df = pd.DataFrame()

for df_piece in range(nb_dataframe_pieces):
    clear_output(wait=True)
    print("processing the piece at position:", df_piece)
    tmp_df = raw_df_pieces[df_piece]
    for var in ['cpg_pos', 'cpg_fm', 'region_inf', 'region_sup']:
        tmp_df.drop(var, axis = 1, inplace = True)
    prepared_df = prepared_df.append(tmp_df)

## Save dataframe with features on Cloud Storage

In [ ]:
# Check the columns
prepared_df.columns

In [ ]:
# Obtain the date/time
now = datetime.today()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
print(dt_string)

In [ ]:
# Export the variable names to the bucket
sys.stdout = open("variables.txt", "w")
%whos

In [ ]:
# Export variable names to to Cloud Storage
dt_string = str(GENOMIC_INTERVAL) + "bp_" + str(NB_ROWS_RAW_DATASET) + "rows_" + dt_string
!gsutil cp variables.txt gs://$DEEPASM_BUCKET/notebook/$dt_string/

In [ ]:
if EXPORT_PROCESSED_DATA == True:
    
    nb_pieces_export = max(1, round(raw_df.shape[0]/2000))
    prepared_df_pieces = np.array_split(prepared_df, nb_pieces_export)
    print("The dataframe has been split into", nb_pieces_export, "pieces")
    
    for df_piece in range(nb_pieces_export):
        print("processing the piece at position:", df_piece)
        df_to_export = prepared_df_pieces[df_piece]
        print("Size of dataframe:", df_to_export.shape)

        print("Saving the file as HDF5...")
        file_name = "prepared_df_" + str(df_piece) + ".h5"
        print("File name:", file_name)
        df_to_export.to_hdf(file_name, key = 'df', mode = 'w')

        print("Exporting file to bucket...")
        !gsutil cp $file_name gs://$DEEPASM_BUCKET/notebook/$dt_string/
else:
        print("Not exporting the scaled DF per variable")

## Importing prepared features from bucket

In [ ]:
if IMPORT_PROCESSED_DATA == True:
    display("Downloading from the bucket...")
    # Obtain path from GCP Cloud Storage
    bucket_path = "gs://" + PROCESSED_DATA_PATH
    file_path = bucket_path + "/*.h5"

    # Find all the H5 files with the same normalization method.
    !gsutil ls $file_path > list_to_download.txt
    files_to_download_df = pd.read_csv('list_to_download.txt', header=None)

    prepared_df = pd.DataFrame()

    for index_file in range(files_to_download_df.shape[0]):
        clear_output(wait=True)
        display("Processing file:", index_file)
        file_name_bucket = bucket_path + "/prepared_df_" + str(index_file) + ".h5"
        display(file_name_bucket)
        file_name_local = "prepared_df_" +  str(index_file) + ".h5"
        !gsutil cp $file_name_bucket $file_name_local
        tmp = pd.read_hdf(file_name_local)
        prepared_df = prepared_df.append(tmp)
else:
    display("Not downloading from the bucket")
    prepared_df = pd.DataFrame()
    !ls *.h5 > list_files.txt
    files_to_append = pd.read_csv('list_files.txt', header=None)
    for file_number in range(files_to_append.shape[0]):
        clear_output(wait=True)
        file_name_local = "prepared_df_" +  str(file_number) + ".h5"
        print("Processing:", file_name_local)
        tmp = pd.read_hdf(file_name_local)
        prepared_df = prepared_df.append(tmp)

In [ ]:
#################################
# TEMPORARY########################
##################################
files_to_append = pd.read_csv('list_files.txt', header=None)
prepared_df = pd.DataFrame()
for file_number in range(files_to_append.shape[0]):
        clear_output(wait=True)
        file_name_local = "prepared_df_" +  str(file_number) + ".h5"
        print("Processing:", file_name_local)
        tmp = pd.read_hdf(file_name_local)
        prepared_df = prepared_df.append(tmp)

In [ ]:
print("Dataset size:", prepared_df.shape)
print("Columns of dataset:", prepared_df.columns)

## Plot a few images

In [ ]:
def add_primary_color(read_info):
    # read_info is a list of 250 arrays. Each subarray is [a,b] where a = 0 or 1 (CpG) and b = 0 or 1 (methylation)
    # We return a list of array with 3 elements. The third one is always zero
    augmented_read = []
    for element in range(0, len(read_info)):
        #print(element)
        # RGB codes: (0,0,0) = black (no CpG), (0,0,1) = dark blue (CpG not methylated) ; 
        # (0,1,1) = light blue (CpG methylated)
        #augmented_read = augmented_read + [[0.0, float(read_info[element][1]), float(read_info[element][0])]]
        
        # RGB codes: (1,1,1) = white (no CpG), (1,0,0) = red (CpG not methylated) [1,0] ; 
        # (0,0,1) = blue (CpG methylated) [1,1]
        if read_info[element][0] == 0: # No CpG
            pixel = [1.0, 1.0, 1.0]
        else:
            if read_info[element][1] == 0: # CpG not methylated
                pixel = [1.0, 0.0, 0.0]
            else:
                pixel = [0.0, 0.0, 1.0]
        augmented_read = augmented_read + [pixel]
    return augmented_read

# Quick test
result = add_primary_color([[0,0],[1,0],[1,1]])
result
#add_primary_color(prepared_df['genomic_2d_picture'][0][0])

In [ ]:
# Black: no CpG
# Dark blue: CpG NOT methylated
# Light blue: CpG METHYLATED

def convert_to_picture(gen2d):
    # gen2d is a list of lists. Each list is structured as above
    
    # Convert into array
    #tmp = np.array(gen2d)
    gen_pic =  [add_primary_color(x) for x in gen2d]
    #gen_pic = map(add_primary_color, gen2d)
    return gen_pic

# Quick test
read1 = [[1,0],[0,0],[1,1], [0,0]]
read2 = [[0,0],[1,0],[1,1], [1,0]]
test = [read1, read2]
result = convert_to_picture(test)
print(result)
imgplot = plt.imshow(result)

In [ ]:
prepared_df

In [ ]:
#prepared_df = prepared_df[len(prepared_df['genomic_2d_picture']) > 0 ]
df_with_asm = prepared_df[prepared_df['asm_snp'] == 1].sample(4).reset_index(drop=True)
df_without_asm = prepared_df[prepared_df['asm_snp'] == 0].sample(4).reset_index(drop=True)

In [ ]:
# PLOT REGIONS WITHOUT ASM
mpl.rcParams['figure.figsize'] = (20, 3)
fig, axs = plt.subplots(2, 2, sharey=False, sharex=False, tight_layout=True)

for k in range(0,2):
    for m in range(0,2):
        sequence = df_without_asm['genomic_2d_picture'][k+m]
        axs[k,m].imshow(convert_to_picture(sequence))

In [ ]:
# PLOT REGIONS WITH ASM
mpl.rcParams['figure.figsize'] = (20, 3)
fig, axs = plt.subplots(2, 2, sharey=False, sharex=False, tight_layout=True)

for k in range(0,2):
    for m in range(0,2):
        sequence = df_with_asm['genomic_2d_picture'][k+m]
        axs[k,m].imshow(convert_to_picture(sequence))

## Class weights

There are approximately 100x more regions without ASM than with ASM. We'll have to use weights in our training.

In [ ]:
neg, pos = np.bincount(prepared_df['asm_snp'])
total = neg + pos
print('Number of regions assessed for ASM: {}\nRegions with ASM found: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight_asm = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

## Normalize the features

In [ ]:
prepared_df.columns

In [ ]:
non_continous_columns = ['asm_snp', 'sample_category', 'dnase', 
                                    'dnase_high', 'dnase_null', 'dnase_low', 
                                    'encode_ChiP_V2_low', 'encode_ChiP_V2_null', 'encode_ChiP_V2_high',
                                    'tf_motifs_null', 'tf_motifs_low', 'tf_motifs_null',
                                    'genomic_matrix']

In [ ]:
# Copy dataframe and remove non-continous variables
normalized_df = prepared_df.copy()
normalized_df = normalized_df.drop(columns = non_continous_columns)

# Apply StandardScaler to continous variables
columns = normalized_df.columns
scaler = StandardScaler()
scaler.fit(normalized_df)
normalized_df = pd.DataFrame(data = scaler.transform(normalized_df), index = normalized_df.index)
normalized_df.columns = columns

# Concatenate with other columns
normalized_df = pd.concat([normalized_df, 
                           prepared_df[non_continous_columns]], axis = 1)

normalized_df

## Perform a PCA to reduce the number of the features to be used in the models

In [ ]:
# # Do a test on normalized features
# features_df = normalized_df.copy()

# # Not normalized features
# #features_df = prepared_df.copy()


In [ ]:
# corr_matrix = pd.DataFrame(abs(features_df.corr()))
# sns.heatmap(corr_matrix)

In [ ]:
# corr_matrix = pd.DataFrame(abs(features_df.corr()['asm_snp'])).sort_values(by = 'asm_snp')
# corr_matrix

### Remove variables with no correlation

In [ ]:
# vars_no_corr = list(corr_matrix[pd.isna(corr_matrix['asm_snp']) == True].index)
# print(vars_no_corr)
# features_df = features_df.drop(columns = vars_no_corr)

### Remove variables that are poorly correlated with ASM

In [ ]:
# vars_low_corr = list(corr_matrix[corr_matrix['asm_snp'] < MIN_CORR].index)
# print(vars_low_corr)
# features_df = features_df.drop(columns = vars_low_corr, axis =1)

### Run a PCA for the remaining variables

In [ ]:
# #Remove the column of genomic matrix and labels
# features_df_pca = features_df.copy()
# features_df_pca = features_df_pca.drop(columns = ['asm_snp', 'genomic_matrix'])
# print("Number of features:", len(features_df_pca.columns))

In [ ]:
# pca = PCA(n_components=10)
# principalComponents = pca.fit_transform(features_df_pca)

In [ ]:
# columns_pca = []
# for k in range(1,11):
#     name = 'pca_' + str(k)
#     columns_pca = columns_pca + [name]
# print(columns_pca)
# features_df_pca = pd.DataFrame(data = principalComponents, columns = columns_pca, index = features_df.index)

In [ ]:
# features_df_pca = pd.concat([features_df_pca, features_df[['asm_snp', 'genomic_matrix']]], axis = 1)
# features_df_pca

In [ ]:
# corr_matrix = pd.DataFrame(abs(features_df_pca.corr()))
# sns.heatmap(corr_matrix)

In [ ]:
# corr_matrix = pd.DataFrame(abs(features_df_pca.corr()['asm_snp'])).sort_values(by = 'asm_snp')
# corr_matrix

## Split the dataset for training and testing

We use the sklearn `train_test_split` function. The validation set will be carved out from the training set when training the model. The validation set will be  used during the model fitting to evaluate the loss and any metrics, however the model is not fit with this data. The test set is completely unused during the training phase and is only used at the end to evaluate how well the model generalizes to new data. 

In [ ]:
# Transpose image


In [ ]:
df_for_split = prepared_df

In [ ]:
#type(df_for_split['genomic_2d_picture'][0])
#df_for_split['genomic_2d_picture'][0]

In [ ]:
train_image_feature, test_image_feature = train_test_split(df_for_split, test_size=TEST_SPLIT)
train_labels = np.array(train_image_feature.pop('asm_snp'))
test_labels = np.array(test_image_feature.pop('asm_snp'))
train_image_feature = np.array(train_image_feature['genomic_2d_picture'].tolist())
test_image_feature = np.array(test_image_feature['genomic_2d_picture'].tolist())
display("Size of the TRAIN dataset for images:", train_image_feature.shape)
display("Size of the TEST dataset for images:", test_image_feature.shape)
display("Size of the TRAIN LABELS dataset:", train_labels.shape)
display("Size of the TEST LABELS dataset:", test_labels.shape)

In [ ]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(df_for_split, test_size=TEST_SPLIT)

# Form np arrays of labels
train_labels = np.array(train_df.pop('asm_snp'))
test_labels = np.array(test_df.pop('asm_snp'))

# Np arrays of features for CNN/RNN
train_image_feature = np.array(train_df['genomic_matrix'].tolist())
test_image_feature = np.array(test_df['genomic_matrix'].tolist())

# Remove the matrix for the datasets
train_df.drop('genomic_matrix', axis = 1, inplace = True)
test_df.drop('genomic_matrix', axis = 1, inplace = True)

# np arrays for  linear/perceptron
train_scalar_features = np.array(train_df)
test_scalar_features = np.array(test_df)

# Check size of arrays for CNN (X,250,3)
display("Image features")
display("Size of the TRAIN dataset for images:", train_image_feature.shape)
display("Size of the TEST dataset for images:", test_image_feature.shape)

# # Check size of arrays for scalar features (X, 39)
display("SCALAR FEATURES:")
display("Size of the TRAIN datase:", train_scalar_features.shape)
display("Size of the TEST dataset:", test_scalar_features.shape)

# # Check size of arrays for the labels
display("LABELS:")
display("Size of the TRAIN LABELS dataset:", train_labels.shape)
display("Size of the TEST LABELS dataset:", test_labels.shape)

## Logistic regression

In [ ]:
def make_logistic_regression_model(output_bias = None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = keras.Sequential()
    
    # Normalize the features
    # model.add(
    #     layers.BatchNormalization(
    #         axis=-1,
    #         momentum = 0.99,
    #         epsilon = 0.001,
    #         input_dim = train_scalar_features.shape[1])
    # )
    model.add(
        layers.Normalization(
            axis = 1,
            input_dim = train_scalar_features.shape[1])
    )
    
    # Linear model
    model.add(
        layers.Dense(
                1,  # number of classes
                activation='sigmoid', #'sigmoid' 'softmax'
                kernel_regularizer = keras.regularizers.L1L2(
                    l1 = L1_R, 
                    l2 = L2_R),
                bias_initializer=output_bias
                )
    )
 
  
    model.compile(
        optimizer = 'sgd' , # sgd = stochastic gradient descent, rmsprop
        loss= 'binary_crossentropy', # 'mse' 'categorical_crossentropy', 'binary_crossentropy'
        metrics = METRICS)

    return model


In [ ]:
# normalizer = layers.Normalization()
# normalizer.adapt(train_scalar_features)

In [ ]:
# # Create a Normalization layer and set its internal state using the training data
# #normalizer = layers.Normalization()
# #normalizer.adapt(train_scalar_features)

# input_shape = train_scalar_features.shape[1:]
# print(input_shape)
# inputs = keras.Input(shape=input_shape)
# x = normalizer(inputs)
# outputs = layers.Dense(1, activation="sigmoid")(x)
# linear_model = keras.Model(inputs, outputs)


In [ ]:
# linear_model.compile(
#         optimizer = 'sgd' , # sgd = stochastic gradient descent, rmsprop
#         loss= 'binary_crossentropy', # 'mse' 'categorical_crossentropy', 'binary_crossentropy'
#         metrics = METRICS)

In [ ]:
linear_model = make_logistic_regression_model()

In [ ]:
linear_model.summary()
keras.utils.plot_model(linear_model, "linear_model.png", show_shapes=True)

In [ ]:
linear_training = linear_model.fit(
    train_scalar_features,
    train_labels,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    callbacks = [EARLY_STOPPING],
    validation_split = VALIDATION_SPLIT,
    class_weight = class_weight_asm,
    verbose = 0) 

In [ ]:
plot_metrics(linear_training)

In [ ]:
# Evaluate model on the test dataset
linear_results = linear_model.evaluate(test_scalar_features, test_labels, batch_size= BATCH_SIZE,
                                           verbose=1)

display_results(linear_results)

## Forest models

### Simple Forest model

In [ ]:
simple_tree_model = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.CLASSIFICATION)

simple_tree_model.compile(metrics=METRICS)

In [ ]:
simple_tree_model.fit(x=train_scalar_features, 
             y = train_labels, 
             batch_size=BATCH_SIZE,
             callbacks = [EARLY_STOPPING],
             class_weight=class_weight_asm, 
             validation_split = VALIDATION_SPLIT,
             verbose = 0)


In [ ]:
simple_tree_results = simple_tree_model.evaluate(
    test_scalar_features, 
    test_labels, 
    batch_size = BATCH_SIZE, 
    return_dict=True)

simple_tree_results

### Gradient Boosted Tree model

In [ ]:
def create_gbt_model():
    boosted_tree_model = tfdf.keras.GradientBoostedTreesModel(
        #features = specify_feature_usages(df_for_split),
        growing_strategy=GROWING_STRATEGY,
        num_trees=NUM_TREES,
        max_depth=MAX_DEPTH,
        min_examples=MIN_EXAMPLES,
        subsample=SUBSAMPLE,
        task=tfdf.keras.Task.CLASSIFICATION,
        loss="DEFAULT",
    )

    boosted_tree_model.compile(metrics=METRICS)
    return boosted_tree_model

In [ ]:
boosted_tree_model = create_gbt_model()

In [ ]:
boosted_tree_model.fit(x = train_scalar_features, 
              y = train_labels, 
              batch_size=BATCH_SIZE, 
              validation_split = VALIDATION_SPLIT,
              callbacks = [EARLY_STOPPING],
              class_weight=class_weight_asm, 
              verbose = 1)

In [ ]:
boosted_tree_results = boosted_tree_model.evaluate(
    test_scalar_features, 
    test_labels, 
    batch_size = BATCH_SIZE, 
    return_dict=True)
boosted_tree_results

## Perceptron model

In [ ]:
def make_perceptron_model(output_bias = None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
  
    model = keras.Sequential()
  
#     # Normalize the features
#     model.add(
#         layers.Normalization(
#             axis = 1,
#             input_dim = train_scalar_features.shape[-1])
#     )
        
  # Initial perceptron layer
    model.add(
        layers.Dense(
            NB_NODES_PERCEPTRON, 
            activation=ACTIVATION_FUNCTION,
            input_dim = train_scalar_features.shape[-1],
            kernel_regularizer = keras.regularizers.L1L2(
                    l1 = L1_R, 
                    l2 = L2_R)
        )
    )
  
    # Range of neuron layers
    for layer_number in range(0, NB_LAYERS_PERCEPTRON-1): 
        model.add(layers.Dense(
            NB_NODES_PERCEPTRON, 
            activation = ACTIVATION_FUNCTION,
            kernel_regularizer = keras.regularizers.L1L2(
                    l1 = L1_R, 
                    l2 = L2_R)))
  
        # Dropout layer in between layers
        model.add(layers.Dropout(0.5))

    # We add a sigmoid to create the probability function of the ASM event.
    model.add(
        layers.Dense(
            1, 
            activation='sigmoid',
            bias_initializer=output_bias,
            kernel_regularizer = keras.regularizers.L1L2(
                    l1 = L1_R, 
                    l2 = L2_R)))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = LEARNING_RATE),
        loss = keras.losses.BinaryCrossentropy(),
        metrics = METRICS)

    return model


In [ ]:
perceptron_model = make_perceptron_model()
perceptron_model.summary()

In [ ]:
perceptron_training = perceptron_model.fit(
    train_scalar_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [EARLY_STOPPING],
    validation_split = VALIDATION_SPLIT,
    class_weight=class_weight_asm,
    verbose = 1) 

In [ ]:
plot_metrics(perceptron_training)

In [ ]:
perceptron_results = perceptron_model.evaluate(test_scalar_features, 
                                               test_labels, 
                                               batch_size= BATCH_SIZE,
                                               verbose=1)
display_results(perceptron_results)

## CNN model

train_image_feature.shape[1], train_image_feature.shape[2]## CNN model with the genomic picture as sole input

In [ ]:
train_image_feature.shape[3]#, train_image_feature.shape[2]

In [ ]:
def make_simple_cnn_model(output_bias = None):

    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)

    # Start the model
    model = keras.Sequential()
    
    # Normalize the features
    # model.add(
    #     layers.BatchNormalization(
    #         axis=-1,
    #         momentum = 0.99,
    #         epsilon = 0.001,
    #         input_shape=(train_image_feature.shape[1], train_image_feature.shape[2])
    #     )
    # )  
    
    # Add a convolutional layer
    model.add(layers.Conv2D(
        filters = 32, 
        kernel_size = (5,5), # (3,3) or (5,5)
        activation = ACTIVATION_FUNCTION,
        input_shape=(train_image_feature.shape[1], train_image_feature.shape[2], train_image_feature.shape[3]),
        kernel_regularizer = keras.regularizers.L1L2(
                            l1 = L1_R, 
                            l2 = L2_R)))

    # Pooling
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
  
    # Flattening
    model.add(layers.Flatten())
  
    # Output layer (Sigmoid)
    model.add(layers.Dense(1, activation='sigmoid', 
                         kernel_regularizer = keras.regularizers.L1L2(
                            l1 = L1_R, 
                            l2 = L2_R),
                         bias_initializer=output_bias))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = LEARNING_RATE),
        loss = 'binary_crossentropy',
        metrics = METRICS)

    return model

In [ ]:
simple_cnn_model = make_simple_cnn_model()

In [ ]:
simple_cnn_model = keras.Sequential()
simple_cnn_model.add(layers.Conv2D(32, (3, 3), input_shape = (20, 250, 2)))
simple_cnn_model.add(layers.Activation('relu'))
simple_cnn_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

simple_cnn_model.add(layers.Conv2D(32, (3, 3)))
simple_cnn_model.add(layers.Activation('relu'))
simple_cnn_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

simple_cnn_model.add(layers.Conv2D(64, (2, 2)))
simple_cnn_model.add(layers.Activation('relu'))
simple_cnn_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

simple_cnn_model.add(layers.Flatten())  # this converts our 3D feature maps to 1D feature vectors
simple_cnn_model.add(layers.Dense(64))
simple_cnn_model.add(layers.Activation('relu'))
simple_cnn_model.add(layers.Dropout(0.5))
simple_cnn_model.add(layers.Dense(1))
simple_cnn_model.add(layers.Activation('sigmoid'))

simple_cnn_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=METRICS)

In [ ]:
# model.summary()
# keras.utils.plot_model(model, "cnn_model.png", show_shapes=True)

In [ ]:
simple_cnn_model.summary()
keras.utils.plot_model(simple_cnn_model, "cnn_model.png", show_shapes=True)

In [ ]:
simple_cnn_training = simple_cnn_model.fit(
    train_image_feature,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [EARLY_STOPPING],
    validation_split = VALIDATION_SPLIT,
    class_weight=class_weight_asm,
    verbose = 1) 

In [ ]:
plot_metrics(simple_cnn_training)

In [ ]:
# Evaluate model on the test dataset
simple_cnn_results = simple_cnn_model.evaluate(
    test_image_feature, 
    test_labels, 
    batch_size = BATCH_SIZE,
    verbose=1)

display_results(simple_cnn_results)

## Recurrent Neural Network

In [ ]:
def make_simple_rnn_model():

    nb_dim_in_genomic_seq = train_image_feature.shape[2] # Should be 3

    model = keras.models.Sequential()
    
    # Normalize the features
    model.add(
        layers.BatchNormalization(
            axis=-1,
            momentum = 0.99,
            epsilon = 0.001,
            input_shape=(train_image_feature.shape[1], train_image_feature.shape[2])
        )
    )  
    
    model.add(layers.LSTM(RNN_UNITS, input_shape=(None, nb_dim_in_genomic_seq), go_backwards = True))

    # Output layer (Sigmoid)
    model.add(layers.Dense(1, activation='sigmoid', 
                         kernel_regularizer = keras.regularizers.L1L2(
                            l1 = L1_R, 
                            l2 = L2_R)
                         ))

    model.compile(
      optimizer = keras.optimizers.Adam(learning_rate =LEARNING_RATE),
      loss = 'binary_crossentropy',
      metrics = METRICS)
    
    return model

In [ ]:
simple_rnn_model = make_simple_rnn_model()

In [ ]:
simple_rnn_model.summary()
keras.utils.plot_model(simple_rnn_model, "cnn_model.png", show_shapes=True)

In [ ]:
simple_rnn_training = simple_rnn_model.fit(
    train_image_feature,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [EARLY_STOPPING],
    validation_split = VALIDATION_SPLIT,
    class_weight=class_weight_asm,
    verbose = 0) 

In [ ]:
plot_metrics(simple_rnn_training)

In [ ]:
# Evaluate model on the test dataset
simple_rnn_results = simple_rnn_model.evaluate(
    test_image_feature, 
    test_labels, 
    batch_size = BATCH_SIZE,
    verbose=1)

display_results(simple_rnn_results)

## Save model results into CSV file

In [ ]:
# MODELS FOR WHICH WE NEED TO RECORD THE RESULTS

models = ['linear', 'perceptron', 'simple_cnn', 'simple_rnn', 'simple_tree', 'boosted_tree']
#models = ['simple_rnn']

# Loss is better than AUC for monitoring
PARAM_TO_CHANGE = "baseline"

In [ ]:
#Initialize DF if it does not exist
try: 
    model_results.head()
    print("does not exist")
except AttributeError:
    pass
except NameError:
    model_results = pd.DataFrame([
                "param_testing", "data_path", "nb_data_points", "genomic_region", "test_split", "val_split", 
                "stop_monitor", "stop_restore_w", 
                "epochs", "batch_size",
                "L1_R", "L2_R",
                'activation_function', 'nb_nodes_perceptron',
                'nb_layers_perceptron', 'nb_nodes_after_cnn',
                'cnn_filters', 'cnn_kernel', 'RNN_units',
                'growing_strat', 'num_trees', 'min_examples', 'max_depth', 'subsample', 'sampling_method',
                'nb_params',
                'loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 
                'sensitivity', 'AUC'], columns = ['parameters'])


In [ ]:
model_results = pd.DataFrame([
                "param_testing", "data_path", "nb_data_points", "genomic_region", "test_split", "val_split", 
                "stop_monitor", "stop_restore_w", 
                "epochs", "batch_size",
                "L1_R", "L2_R",
                'activation_function', 'nb_nodes_perceptron',
                'nb_layers_perceptron', 'nb_nodes_after_cnn',
                'cnn_filters', 'cnn_kernel', 'RNN_units',
                'growing_strat', 'num_trees', 'min_examples', 'max_depth', 'subsample', 'sampling_method',
                'nb_params',
                'loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 
                'sensitivity', 'AUC'], columns = ['parameters'])

In [ ]:
for model in models:
  print("model:", model) 
  name_results = model + "_results"
  name_model = model + "_model"

  command_nb_params = "nb_params = " + name_model + ".count_params()"
  exec(command_nb_params)

  # Parameters common to all models
  common_param = pd.DataFrame([
     PARAM_TO_CHANGE,
     PROCESSED_DATA_PATH,
     normalized_df.shape[0],
     GENOMIC_INTERVAL,
      TEST_SPLIT,
     VALIDATION_SPLIT,
     EARLY_STOPPING.monitor,
     EARLY_STOPPING.restore_best_weights,
     EPOCHS,
     BATCH_SIZE,
     L1_R,
     L2_R,
     ACTIVATION_FUNCTION,
     NB_NODES_PERCEPTRON,
     NB_LAYERS_PERCEPTRON,
     NB_NODES_AFTER_CNN,
     CNN_FILTERS,
     CNN_KERNEL,
     RNN_UNITS,
     GROWING_STRATEGY,
     NUM_TREES,
     MIN_EXAMPLES,
     MAX_DEPTH,
     SUBSAMPLE,
     SAMPLING_METHOD,
     nb_params])

  # Create dataframe from the model results
  if "tree" in model:
      #print("Found a tree")
      command_df_new_results = "tmp = pd.DataFrame(np.round(list(" + name_results + ".values()), 3))"
  else:
      #print("Not a tree")
      command_df_new_results = "tmp = pd.DataFrame(np.round(" + name_results + ", 3))"
  exec(command_df_new_results)
  #print(tmp)

  # Append the two dataframes
  new_column = common_param.append(tmp, ignore_index = True)

  # Rename the dataframe
  new_column.columns = [model]
  #print(new_column)

  # Add the results to the dataframe of results
  model_results = model_results.merge(new_column, left_index = True, right_index = True)
 
model_results

In [ ]:
model_results.to_csv('model_results.csv', index=False)

In [ ]:
simple_tree_model.count_params()

In [ ]:
linear_results

In [ ]:
list(simple_tree_results.values())

In [ ]:
tmp = pd.DataFrame(np.round(simple_tree_results, 3))